In [1]:
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Pac

In [2]:
import time
import pandas as pd

# Импорт Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

def setup_driver():
    """Настройка headless Chrome (без интерфейса)"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # В Colab chromedriver обычно находится в /usr/bin/chromedriver
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def parse_table(driver):
    """
    Извлекает данные со страницы, возвращает список записей:
      [ [№ Участника, Наименование, БИН, ИИН, РНН], ... ]
    """
    data = []
    # Находим все строки таблицы
    rows = driver.find_elements(By.CSS_SELECTOR, "table.table-bordered tbody tr")
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) >= 5:
            reg_number = cols[0].text.strip()
            company_name = cols[1].text.strip()

            bin_number = cols[2].text.strip()
            iin = cols[3].text.strip()
            rnn = cols[4].text.strip()

            data.append([
                reg_number,
                company_name,
                bin_number,
                iin,
                rnn
            ])
    return data

def main():
    # Стартовая страница (с «пустыми» фильтрами)
    start_url = "https://fms.ecc.kz/ru/register/supplierreg?name_bin_iin_rnn=&country=&region_supplier="

    driver = setup_driver()
    driver.get(start_url)
    time.sleep(2)  # Небольшая пауза, чтобы страница успела загрузиться

    all_data = []
    visited_urls = set()

    while True:
        current_url = driver.current_url
        if current_url in visited_urls:
            # Цикл: сайт не даёт новых страниц
            break
        visited_urls.add(current_url)

        # Парсим таблицу
        page_data = parse_table(driver)
        all_data.extend(page_data)

        # Ищем кнопку ">"
        try:
            pagination = driver.find_element(By.CSS_SELECTOR, "ul.pagination")
            arrow = pagination.find_element(By.XPATH, ".//a[text()='>']")
            # Пробуем кликнуть
            arrow.click()
            time.sleep(2)  # Ждём загрузки следующей страницы
        except NoSuchElementException:
            print("Кнопка '>' не найдена, заканчиваем.")
            break
        except ElementNotInteractableException:
            print("Кнопка '>' не активна, заканчиваем.")
            break

    # Закрываем браузер
    driver.quit()

    # Формируем DataFrame
    df = pd.DataFrame(all_data, columns=[
        "№ Участника",
        "Наименование участника",
        "БИН (ИНН, УНП)",
        "ИИН",
        "РНН"
    ])

    # Сохраняем в CSV и Excel
    df.to_csv("suppliers.csv", index=False, encoding="utf-8-sig")
    df.to_excel("suppliers.xlsx", index=False)
    print(f"Всего спарсили записей: {len(df)}")
    print("✅ Данные сохранены в suppliers.csv и suppliers.xlsx")

    return df

if __name__ == "__main__":
    df = main()
    # Если хотите посмотреть в Colab, раскомментируйте:
from IPython.display import display
display(df)

Кнопка '>' не найдена, заканчиваем.
Всего спарсили записей: 5447
✅ Данные сохранены в suppliers.csv и suppliers.xlsx


,№ Участника,Наименование участника,"БИН (ИНН, УНП)",ИИН,РНН
0,397315,"""DOFAMIN CLINIC"" жауапкершілігі шектеулі серік...",241240000503,,
1,397275,"Товарищество с ограниченной ответственностью ""...",240140015654,,
2,397235,"""НұрТұрсын"" жауапкершілігі шектеулі серіктестігі",001240004309,,
3,397135,"Товарищество с ограниченной ответственностью ""...",180140026130,,
4,397075,"ТОО ""CK Health center""",241240010997,,
...,...,...,...,...,...
5442,109787,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",930340000390,,
5443,109767,"ТОО ""КФК""",051140005015,,
5444,109727,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",020340001180,,
5445,109707,"""САДЫХАН ПРЕМИУМ"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІ...",061240010700,,
